In [1]:
import pandas as pd
import seaborn as sns
import janitor
sns.set()

In [2]:
# data import and standardizing var names with pyjanitor
demo_phq = pd.read_csv("../data/raw/DEMO_PHQ.csv").clean_names()
pag_hei = pd.read_csv("../data/raw/PAG_HEI.csv").clean_names()

# Data Dictionary

## DEMO_PHQ

- **seqn**: unique identifier (respondent)
- **dpq0_**: are those answers for the phq_9 form, values from 0 to 3 on each column are expected
- **riagendr**: gender
<span style="color:yellow">
  - 1: male
  - 2: female
</span>
- **ridageyr**: age in years
- **ridreth1**: race
<span style="color:yellow">
  - 1: white non-hispanic
  - 2: black non-hispanic
  - 3: mexican-american
  - 4: other
  - 5: other-hispanic
</span>
- **dmdeduc**: schooling
<span style="color:yellow">
  - 1: miner than 9 year
  - 2: 9 to 12 year
  - 3: Middle school
  - 4: Supirior uncomplished
  - 5: Full supirior
  - 7: not want to answer
  - 9: don't know
</span>
- **indfminc**: anual familiar revenue in US$
<span style="color:yellow">
  - 1: 0 - 4999
  - 2: 5000 - 9999
  - 3: 10000 - 14999
  - 4: 15000 - 19999
  - 5: 20000 - 24999
  - 6: 25000 - 34999
  - 7: 35000 - 44999
  - 8: 45000 - 54999
  - 9: 55000 - 64999
  - 10: 65000 - 74999
  - 11: >= 75000
  - 12: > 20000
  - 13: < 20000
  - 77: don't want to answer
  - 99: don't know
</span>


## PAG_HEI

- seqn: respondent unique identifier
- pag_minw: weekly total of aerobic activity moderetly-vigorous in min (PAG)
- adherence: adherence group:
<span style="color:yellow">
  - 1: low (< 150 min/week)
  - 2: adjusted (150 - 300 min/week)
  - 3: up (> 300 min/week)
</span>
- hei2015_: health eating indexes, those ranges are:
<span style="color:yellow">
  - 0-5: vegetables, dark green vegetables and bean, fruits, in nature fruits, proteins, sea plant and protein plant.
  - 0-10: whole beans, dairy, fatty acid, sodium, refined grain, satured fat, add sugger
  - 0-100: final score 
</span>

# Data Pre-Processing

In [3]:
pag_hei_seqn_ids = pag_hei.seqn.to_list()
demo_phq_seqn_ids = demo_phq.seqn.to_list()

# comparing lists
pag_hei_to_demo_phq = set(pag_hei_seqn_ids) - set(demo_phq_seqn_ids)
print(len(pag_hei_to_demo_phq))

4090


**Questions**: 
- Why do we have 4090 respondents that are present on HEI and not in PHQ_9?
- How those unique identifiers are correlated to each other?

## Merging the data


In [7]:
demo_phq_pag_hei_df = pd.merge(
    left=demo_phq, 
    right=pag_hei, 
    on="seqn"
)


## Calculating the PHQ_9 index

In [46]:
# values for phq_9 score
phq_9_score = demo_phq_pag_hei_df.iloc[:, 1:10].sum(axis=1)

df = demo_phq_pag_hei_df.add_column("phq_9_score", phq_9_score)
df["phq_9_score_str"] = None

# applying conditionals for each phq_9 score
df.loc[df.phq_9_score.between(0, 4), "phq_9_score_str"] = "Minimal depression"
df.loc[df.phq_9_score.between(5, 9), "phq_9_score_str"] = "Mild depression"
df.loc[df.phq_9_score.between(10, 14), "phq_9_score_str"] = "Moderate depression"
df.loc[df.phq_9_score.between(15, 19), "phq_9_score_str"] = "Moderately severe depression"
df.loc[df.phq_9_score.between(20, 27), "phq_9_score_str"] = "Severe depression"

# identifying where all the answers was missing
rows = df.shape[0]
rows_list = [df.iloc[r, 1:10].to_list() for r in range(rows)]

# creating a mask to use as filter
rows_bools = [all(i) for i in rows_list]

df.loc[rows_bools, "phq_9_score_str"] = None


In our dataset we saw that some respondents did not answer those questions about the phq_9, I'm assuming that we cannot assume 0 to those cases, instead we're putting a missing value on `phq_9_score_str` column. I prefer do not replace all numerical representations of categorical variables yeat, because this can be very time consuming, instead, I do that when we goes to create some visualizations on EDA.

In [52]:
# exporting the dataframe to use on EDA
df.to_csv("../data/curated/demo_phq_pag_hei_cureted.csv", index=False)